In [215]:
import glob
import pandas as pd

In [217]:
# Local copy of Illumina QC
illumina_sample_df = pd.DataFrame(glob.glob('/private/groups/hprc/1000G_local/child/fastq/*.fq.gz'))
illumina_sample_df['sample_id'] = [sample.split('/')[-1].split('.')[0] for sample in illumina_sample_df[0].tolist()]
print(illumina_sample_df.shape)

(171, 2)


In [218]:
# AWS Submission
illumina_y1_y4 = pd.read_csv('Illumina_Y1-Y4.csv',header=None)
illumina_y1_y4.head()
illumina_y1_y4 = illumina_y1_y4[illumina_y1_y4[0].str.contains('final.cram') & ~illumina_y1_y4[0].str.contains('.md5') & ~illumina_y1_y4[0].str.contains('.cram.crai')]
illumina_y1_y4['sample_id'] = [sample.split('/')[-1].split('.')[0] for sample in illumina_y1_y4[0].tolist()]
print(illumina_y1_y4.shape)

(108, 2)


In [130]:
# len([sample for sample in illumina_sample_df.sample_id.tolist() if sample  not in illumina_y1_y4.sample_id.tolist()])

In [121]:
# # Check if all values in the list are in the column and if the number of values matches the DataFrame length
# set(illumina_sample_df['sample_id'].tolist()).issubset(set(illumina_y1_y4['sample_id'])) and len(illumina_sample_df['sample_id'].tolist()) == len(illumina_y1_y4)

In [219]:
# AWS Submission
illumina_y4 = pd.read_csv('Y4_1000G_DATA.csv', header=None)
illumina_y4['sample_id'] = [sample.split('/')[-1].split('.')[0] for sample in illumina_y4[0].tolist()]
print(illumina_y4.shape)

(108, 2)


In [133]:
# assert illumina_y4[illumina_y4['sample_id'].isin(illumina_sample_df.sample_id.tolist())].shape[0] == 0

In [227]:
# concatenate aws submission tables
illumina_aws_submission = pd.concat([illumina_y1_y4, illumina_y4])
illumina_aws_submission['location'] = 'aws'
illumina_sample_df['location'] = 'local'

In [228]:
illumina_df = pd.concat([illumina_aws_submission, illumina_sample_df])
print(illumina_df.shape)

(387, 3)


In [229]:
# subset by duplicated sample_id and keep the aws location
test = illumina_df[illumina_df.duplicated(subset='sample_id', keep=False)]
test = test[test['location'].isin(['aws'])]

In [231]:
illumina_df = illumina_df[~illumina_df['sample_id'].isin(test['sample_id'].tolist())]

illumina_final_df = pd.concat([illumina_df, test])

In [237]:
set(illumina_final_df.location.tolist())

{'aws', 'local'}

In [202]:
test[test['sample_id'].isin(['HG02027'])]

,0,sample_id,location
82,s3://human-pangenomics/submissions/325b4b1c-9f...,HG02027,aws


In [197]:
test

,0,sample_id,location
1,s3://human-pangenomics/submissions/325b4b1c-9f...,HG00099,aws
3,s3://human-pangenomics/submissions/325b4b1c-9f...,HG00140,aws
5,s3://human-pangenomics/submissions/325b4b1c-9f...,HG00280,aws
7,s3://human-pangenomics/submissions/325b4b1c-9f...,HG00323,aws
9,s3://human-pangenomics/submissions/325b4b1c-9f...,HG00423,aws
...,...,...,...
276,s3://human-pangenomics/submissions/325b4b1c-9f...,NA20129,aws
279,s3://human-pangenomics/submissions/325b4b1c-9f...,NA20752,aws
281,s3://human-pangenomics/submissions/325b4b1c-9f...,NA20799,aws
283,s3://human-pangenomics/submissions/325b4b1c-9f...,NA20805,aws


In [ ]:
# Sort the DataFrame so that 'aws' values appear first for each 'sample_id'
df_sorted = df.sort_values(by=['sample_id', 'location'], ascending=[True, False])

# Drop duplicates based on 'sample_id', keeping the first (which will be 'aws' if present)
df_unique = df_sorted.drop_duplicates(subset='sample_id', keep='first')


(387, 3)

In [152]:
pd.concat([illumina_aws_submission, illumina_sample_df]).sample_id.nunique()

279

In [144]:
len([sample for sample in illumina_aws_submission.sample_id.tolist() if sample not in illumina_sample_df['sample_id'].tolist()])

108

In [143]:
len([sample for sample in illumina_sample_df.sample_id.tolist() if sample not in illumina_aws_submission['sample_id'].tolist()])

63

In [110]:
illumina_sample_df[~illumina_sample_df['sample_id'].isin(illumina_aws_submission['sample_id'].tolist())].shape

(63, 2)

In [103]:
illumina_aws_submission[illumina_aws_submission['sample_id'].isin(illumina_sample_df['sample_id'].tolist())].shape

(108, 2)

In [53]:
illumina_y1_y4[~illumina_y1_y4['sample_id'].isin(illumina_sample_df['sample_id'].tolist())]

,0,sample_id


In [52]:
illumina_y1_y4[illumina_y1_y4['sample_id'].isin(illumina_sample_df['sample_id'].tolist())].shape[0] == illumina_y1_y4.shape[0]

True

In [29]:
# Number of Illumina samples
illumina_sample_df.shape[0] + illumina_y4.shape[0]

279

In [5]:
illumina_sample_df.rename(columns={0:'illumina_reads'},inplace=True)

In [12]:
illumina_sample_df[['sample_id',
					'illumina_reads']].to_csv('illumina_qc/illumina_sample_files.csv',index=False)